In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

import sys
import os
import pytreebank
import nltk
import re


from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics import f1_score, accuracy_score
from textblob import TextBlob
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
import string

from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.python.keras.callbacks import EarlyStopping

### LOADING THE DATASET

In [2]:
data = pytreebank.load_sst("data/SST2-Data/SST2-Data/trainDevTestTrees_PTB/trees/")

In [3]:
out_path = os.path.join("data/sst_{}.txt")

In [4]:
for cat in ['train','test','dev']:
    with open(out_path.format(cat),"w") as file:
        for item in data[cat]:
            file.write("__label__{}\t{}\n".format(
                item.to_labeled_lines()[0][0] +1,
                item.to_labeled_lines()[0][1]
            ))
    
    print("done with {}".format(file))

done with <_io.TextIOWrapper name='data/sst_train.txt' mode='w' encoding='UTF-8'>
done with <_io.TextIOWrapper name='data/sst_test.txt' mode='w' encoding='UTF-8'>
done with <_io.TextIOWrapper name='data/sst_dev.txt' mode='w' encoding='UTF-8'>


In [5]:
df_train = pd.read_csv("data/sst_train.txt",sep="\t",header=None,names=['label','text'])
df_train['label'] = df_train['label'].str.replace("__label__","")
# df_train['label'] = df_train['label'].astype(int).astype("category")
df_train['label'] = (df_train['label'].astype(int) - 1).astype('category')

df_test = pd.read_csv("data/sst_test.txt",sep="\t",header=None,names=['label','text'])
df_test['label'] = df_test['label'].str.replace("__label__","")
# df_test['label'] = df_test['label'].astype(int).astype("category")
df_test['label'] = (df_test['label'].astype(int) - 1).astype('category')

In [39]:
df_train

,label,text
0,3,The Rock is destined to be the 21st Century 's...
1,4,The gorgeously elaborate continuation of `` Th...
2,3,Singer/composer Bryan Adams contributes a slew...
3,2,You 'd think by now America would have had eno...
4,3,Yet the act is still charming here .
...,...,...
8539,0,A real snooze .
8540,1,No surprises .
8541,3,We 've seen the hippie-turned-yuppie plot befo...
8542,0,Her fans walked out muttering words like `` ho...


In [40]:
df_test

,label,text
0,2,Effective but too-tepid biopic
1,3,If you sometimes like to go to the movies to h...
2,4,"Emerges as something rare , an issue movie tha..."
3,2,The film provides some great insight into the ...
4,4,Offers that rare combination of entertainment ...
...,...,...
2205,3,An imaginative comedy/thriller .
2206,4,"( A ) rare , beautiful film ."
2207,4,( An ) hilarious romantic comedy .
2208,3,Never ( sinks ) into exploitation .


### DATA PREPROCESSING

In [6]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def preprocess_text(text):
    text = text.lower()
    return text

def remove_punctuation(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

def remove_stopwords(text):
    stop_words = set(stopwords.words("english"))
    filtered_text = [word for word in text.split() if word not in stop_words]
    return " ".join(filtered_text)

In [7]:
df_train['text'] = df_train['text'].apply(strip_html)
df_train['text'] = df_train['text'].apply(preprocess_text)
df_train['text'] = df_train['text'].apply(remove_punctuation)
df_train['text'] = df_train['text'].apply(remove_stopwords)

/var/folders/0s/6y_9xhvn7tx64mnfljyvm01m0000gn/T/ipykernel_80946/3674060142.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [8]:
df_test['text'] = df_test['text'].apply(strip_html)
df_test['text'] = df_test['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(remove_punctuation)
df_test['text'] = df_test['text'].apply(remove_stopwords)

/var/folders/0s/6y_9xhvn7tx64mnfljyvm01m0000gn/T/ipykernel_80946/3674060142.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


### MULTI-LAYER PERCEPTRON (MLP)

In [10]:
train_dataset = tf.data.Dataset.from_tensor_slices((df_train['text'], df_train['label']))
test_dataset = tf.data.Dataset.from_tensor_slices((df_test['text'], df_test['label']))

#### Google News Corpus

In [17]:
embedding = "https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=False)  # You can set trainable to True or False based on your needs

model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(5, activation='softmax'))

model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])


# Batch the datasets
batch_size = 32
train_dataset = train_dataset.batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    train_dataset,      
    epochs=50,                                 
    validation_data=test_dataset, 
    verbose=1,
    callbacks=[early_stopping]
)


Epoch 1/50
  1/267 [..............................] - ETA: 51s - loss: 1.5873 - accuracy: 0.2188

/Users/mazeltan/Desktop/CZ4042_Group_project/env/lib/python3.11/site-packages/keras/src/backend.py:5729: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


267/267 [==============================] - 1s 2ms/step - loss: 1.4621 - accuracy: 0.3817 - val_loss: 1.7629 - val_accuracy: 0.2878
Epoch 2/50
267/267 [==============================] - 0s 891us/step - loss: 1.4547 - accuracy: 0.3749 - val_loss: 1.6380 - val_accuracy: 0.2959
Epoch 3/50
267/267 [==============================] - 0s 874us/step - loss: 1.3914 - accuracy: 0.3940 - val_loss: 1.6015 - val_accuracy: 0.3086
Epoch 4/50
267/267 [==============================] - 0s 899us/step - loss: 1.3531 - accuracy: 0.4127 - val_loss: 1.5750 - val_accuracy: 0.3240
Epoch 5/50
267/267 [==============================] - 0s 875us/step - loss: 1.3301 - accuracy: 0.4215 - val_loss: 1.5573 - val_accuracy: 0.3348
Epoch 6/50
267/267 [==============================] - 0s 852us/step - loss: 1.3157 - accuracy: 0.4292 - val_loss: 1.5463 - val_accuracy: 0.3462
Epoch 7/50
267/267 [==============================] - 0s 839us/step - loss: 1.3063 - accuracy: 0.4371 - val_loss: 1.5376 - val_accuracy: 0.3511
Epoch

#### Wikipedia Corpus

In [11]:
embedding = "https://tfhub.dev/google/Wiki-words-250-with-normalization/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=False)  # You can set trainable to True or False based on your needs

model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(5, activation='softmax'))


model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

# train_dataset = tf.data.Dataset.from_tensor_slices((df_train['text'], df_train['label']))
# test_dataset = tf.data.Dataset.from_tensor_slices((df_test['text'], df_test['label']))

batch_size = 32
train_dataset = train_dataset.batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    train_dataset,      
    epochs=50,                                 
    validation_data=test_dataset, 
    verbose=1,
    callbacks=[early_stopping]
)


Epoch 1/50
  1/267 [..............................] - ETA: 51s - loss: 1.6357 - accuracy: 0.0625

/Users/mazeltan/Desktop/CZ4042_Group_project/env/lib/python3.11/site-packages/keras/src/backend.py:5729: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


267/267 [==============================] - 1s 3ms/step - loss: 1.4519 - accuracy: 0.4155 - val_loss: 1.6886 - val_accuracy: 0.2878
Epoch 2/50
267/267 [==============================] - 0s 2ms/step - loss: 1.4901 - accuracy: 0.3779 - val_loss: 1.6342 - val_accuracy: 0.2919
Epoch 3/50
267/267 [==============================] - 0s 607us/step - loss: 1.4532 - accuracy: 0.3857 - val_loss: 1.6418 - val_accuracy: 0.2977
Epoch 4/50
267/267 [==============================] - 0s 617us/step - loss: 1.4155 - accuracy: 0.4088 - val_loss: 1.6999 - val_accuracy: 0.2982
Epoch 5/50
267/267 [==============================] - 0s 606us/step - loss: 1.3814 - accuracy: 0.4068 - val_loss: 1.8050 - val_accuracy: 0.2950
Epoch 6/50
267/267 [==============================] - 0s 613us/step - loss: 1.3751 - accuracy: 0.4230 - val_loss: 1.6724 - val_accuracy: 0.2995
Epoch 7/50
267/267 [==============================] - 1s 2ms/step - loss: 1.3631 - accuracy: 0.4240 - val_loss: 1.6771 - val_accuracy: 0.3104
